#Setup

In [ ]:
import pandas as pd # Data handling
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Mount drive (if data is kept on Google Drive)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# For turning categorical variables into a vector
le = LabelEncoder()

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# Initialises the logistic regression
model = LogisticRegression(solver='lbfgs')

# To standardise embeddings (mean=0, std=1) - with z score
scaler = StandardScaler()

#Import df and init x and y

In [ ]:
#reddit 105:
reddit_105= pd.read_parquet("/content/drive/MyDrive/STAR Research/Reddit 2 author tests/reddit_105.parquet")
# Unravel the embeddings in the 'embedding' column into a matrix
X = np.stack(reddit_105['embedding'])

# fit assigns a unique integer to each author
# transform actually converts the strings to a NumPy array with those integers
y = le.fit_transform(reddit_105['author'])

In [ ]:
#reddit 921:
reddit_921= pd.read_parquet("/content/drive/MyDrive/STAR Research/Reddit 2 author tests/reddit_921.parquet")
# Unravel the embeddings in the 'embedding' column into a matrix
X = np.stack(reddit_921['embedding'])

# fit assigns a unique integer to each author
# transform actually converts the strings to a NumPy array with those integers
y = le.fit_transform(reddit_921['author'])

In [ ]:
#no quote reddit 921:
nq_reddit_921= pd.read_parquet("/content/drive/MyDrive/reddit_921_plus_emb.parquet")
# Unravel the embeddings in the 'embedding' column into a matrix
X = np.stack(nq_reddit_921['embedding'])

# fit assigns a unique integer to each author
# transform actually converts the strings to a NumPy array with those integers
y = le.fit_transform(nq_reddit_921['author'])

#logreg

In [ ]:
all_folds_preds = []
all_folds_true = []

for train_idx, val_idx in skf.split(X, y):
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    model = LogisticRegression(solver='lbfgs', max_iter=3000)
    model.fit(X_train, y_train)

    val_preds=model.predict(X_val)
    all_folds_preds.extend(val_preds)
    all_folds_true.extend(y_val)
print("accuracy:", accuracy_score(all_folds_true, all_folds_preds))

accuracy: 1.0


#Interpret weights

In [ ]:
abs_weights=abs(model.coef_)#here rows are authors, columns are dimensions
abs_weights_df=pd.DataFrame(abs_weights)
abs_weights_df.to_parquet(f'/content/drive/MyDrive/STAR Research/reddit_921_nq_weights.parquet')

In [ ]:
print(np.max(abs_weights))
print(np.argmax(abs_weights))

0.017888874830305247
361
